# Detecting bad posture

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
import numpy as np

2022-01-13 23:01:01.420904: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-13 23:01:01.420926: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
BATCH_SIZE = 16 # 10, 8, 16
ROTATION = 0.05

In [3]:
idg = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input, validation_split=0.1)
train = idg.flow_from_directory(directory="../img/train",
                                shuffle=True,
                                target_size=(224, 224), 
                                classes=["good", "bad"], 
                                batch_size=BATCH_SIZE,
                                subset='training')

val = idg.flow_from_directory(directory="../img/train",
                                target_size=(224, 224), 
                                classes=["good", "bad"], 
                                batch_size=BATCH_SIZE,
                                subset='validation')

Found 1045 images belonging to 2 classes.
Found 115 images belonging to 2 classes.


In [4]:
mobile_model = tf.keras.applications.MobileNetV2()

2022-01-13 23:01:03.578528: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-13 23:01:03.585993: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-13 23:01:03.640078: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-13 23:01:03.640365: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1650 Ti with Max-Q Design computeCapability: 7.5
coreClock: 1.2GHz coreCount: 16 deviceMemorySize: 3.82GiB deviceMemoryBandwidth: 149.04GiB/s
2022-01-13 23:01:03.640477: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file

In [5]:
def count_params(model):
    non_trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.non_trainable_weights])
    trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.trainable_weights])
    return {"non_trainable_params": non_trainable_params, "trainable_params": trainable_params}

In [6]:
params = count_params(mobile_model)
print(params)

{'non_trainable_params': 34112, 'trainable_params': 3504872}


In [8]:
for layer in mobile_model.layers[:100]:
    layer.trainable = False

In [9]:
model = Sequential()
model.add(mobile_model)
model.add(Flatten())
model.add(Dense(units=2, activation="softmax"))

In [10]:
print(count_params(model))

{'non_trainable_params': 396544, 'trainable_params': 3144442}


In [11]:
# Prove on decreasing learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [12]:
model.fit(x=train, validation_data=val, epochs=5, shuffle=True, class_weight={0:0.2, 1:0.8})

2022-01-13 23:03:25.800115: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-13 23:03:25.819646: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2400000000 Hz


Epoch 1/5
66/66 [==============================] - 28s 392ms/step - loss: 0.2021 - precision: 0.8857 - recall: 0.8857 - val_loss: 0.6502 - val_precision: 0.8609 - val_recall: 0.8609
Epoch 2/5
66/66 [==============================] - 27s 407ms/step - loss: 0.1797 - precision: 0.9738 - recall: 0.9738 - val_loss: 0.6400 - val_precision: 0.8696 - val_recall: 0.8696
Epoch 3/5
66/66 [==============================] - 27s 406ms/step - loss: 0.1871 - precision: 0.9813 - recall: 0.9813 - val_loss: 0.6288 - val_precision: 0.8870 - val_recall: 0.8870
Epoch 4/5
66/66 [==============================] - 27s 407ms/step - loss: 0.1859 - precision: 0.9880 - recall: 0.9880 - val_loss: 0.6201 - val_precision: 0.8870 - val_recall: 0.8870
Epoch 5/5
66/66 [==============================] - 27s 409ms/step - loss: 0.1819 - precision: 0.9871 - recall: 0.9871 - val_loss: 0.6106 - val_precision: 0.9043 - val_recall: 0.9043


In [13]:
model.save("../models/mobile.h5")